In [4]:
import pandas as pd

In [5]:
pd.read_csv('test1.csv')

,user,time_stamp,btag,cate,brand
0,558157,1493741625,pv,6250,91286
1,558157,1493741626,pv,6250,91286
2,558157,1493741627,pv,6250,91286
3,728690,1493776998,pv,11800,62353
4,332634,1493809895,pv,1101,365477
...,...,...,...,...,...
9999995,80919,1493534462,pv,8071,452272
9999996,80919,1493534468,pv,8071,452272
9999997,80919,1493534468,pv,8071,452272
9999998,80919,1493524652,pv,8071,452272


In [6]:
reader = pd.read_csv('test1.csv',chunksize=100,iterator=True)
count = 0;
for chunk in reader:
    count += 1
    if count ==1:
        chunk.to_csv('test5.csv',index = False)
    elif count>1 and count<100:
        chunk.to_csv('test5.csv',index = False, mode = 'a',header = False)
    else:
        break
pd.read_csv('test5.csv')

,user,time_stamp,btag,cate,brand
0,558157,1493741625,pv,6250,91286
1,558157,1493741626,pv,6250,91286
2,558157,1493741627,pv,6250,91286
3,728690,1493776998,pv,11800,62353
4,332634,1493809895,pv,1101,365477
...,...,...,...,...,...
9895,207625,1493820113,pv,4546,264740
9896,207625,1493820121,pv,4546,264740
9897,207625,1493820087,pv,4546,264740
9898,525807,1493784044,pv,4546,264740


In [9]:
import os
# 配置spark driver和pyspark运行时，所使用的python解释器路径
PYSPARK_PYTHON = "/miniconda2/envs/py365/bin/python"
JAVA_HOME='/root/bigdata/jdk'
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON
os.environ['JAVA_HOME']=JAVA_HOME

# spark配置信息
from pyspark import SparkConf
from pyspark.sql import SparkSession

SPARK_APP_NAME = "ALSRecommend"
SPARK_URL = "spark://192.168.247.111:7077"

conf = SparkConf()    # 创建spark config对象
config = (
	("spark.app.name", SPARK_APP_NAME),    # 设置启动的spark的app名称，没有提供，将随机产生一个名称
	("spark.executor.memory", "6g"),    # 设置该app启动时占用的内存用量，默认1g
	("spark.master", SPARK_URL),    # spark master的地址
    ("spark.executor.cores", "2"),    # 设置spark executor使用的CPU核心数
)
# 查看更详细配置及说明：https://spark.apache.org/docs/latest/configuration.html

conf.setAll(config)

# 利用config对象，创建spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [2]:
behavior_log_df = spark.read.csv('/data/test1.csv',header = True)

In [3]:
behavior_log_df.show()

+------+----------+----+-----+------+
|  user|time_stamp|btag| cate| brand|
+------+----------+----+-----+------+
|558157|1493741625|  pv| 6250| 91286|
|558157|1493741626|  pv| 6250| 91286|
|558157|1493741627|  pv| 6250| 91286|
|728690|1493776998|  pv|11800| 62353|
|332634|1493809895|  pv| 1101|365477|
|857237|1493816945|  pv| 1043|110616|
|619381|1493774638|  pv|  385|428950|
|467042|1493772641|  pv| 8237|301299|
|467042|1493772644|  pv| 8237|301299|
|991528|1493780710|  pv| 7270|274795|
|991528|1493780712|  pv| 7270|274795|
|991528|1493780712|  pv| 7270|274795|
|991528|1493780712|  pv| 7270|274795|
|991528|1493780714|  pv| 7270|274795|
|991528|1493780765|  pv| 7270|274795|
|991528|1493780714|  pv| 7270|274795|
|991528|1493780765|  pv| 7270|274795|
|991528|1493780764|  pv| 7270|274795|
|991528|1493780633|  pv| 7270|274795|
|991528|1493780764|  pv| 7270|274795|
+------+----------+----+-----+------+
only showing top 20 rows



In [4]:
behavior_log_df.printSchema()

root
 |-- user: string (nullable = true)
 |-- time_stamp: string (nullable = true)
 |-- btag: string (nullable = true)
 |-- cate: string (nullable = true)
 |-- brand: string (nullable = true)



In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
# 构建结构对象
schema = StructType([
    StructField("userId", IntegerType()),
    StructField("timestamp", LongType()),
    StructField("btag", StringType()),
    StructField("cateId", IntegerType()),
    StructField("brandId", IntegerType())
])
# 从hdfs加载数据为dataframe，并设置结构
behavior_log_df = spark.read.csv("/data/test1.csv", header=True, schema=schema)
#behavior_log_df.show()
behavior_log_df.count() 

10000000

In [6]:
behavior_log_df.groupBy("userId").count().show()

+-------+-----+
| userId|count|
+-------+-----+
|  57178|   54|
| 747610|    8|
|1081474|    3|
| 694901|   10|
| 920750|   70|
|  35361|   16|
| 961035|    5|
| 155430|    6|
| 493421|   49|
| 897491|    4|
|  47217|   18|
| 976417|   28|
| 295048|    7|
| 876272|   26|
| 495753|    8|
| 691528|   52|
|1131890|   98|
| 763546|    1|
| 753966|    6|
| 300779|    9|
+-------+-----+
only showing top 20 rows



In [9]:
#第一次count是针对每个用户行为有多少次，第二次count是统计多少用户的
behavior_log_df.groupBy("userId").count().count()

745163

In [10]:
behavior_log_df.groupBy('btag').count().collect()

[Row(btag='buy', count=130132),
 Row(btag='fav', count=129493),
 Row(btag='cart', count=220382),
 Row(btag='pv', count=9519993)]

In [7]:
13/950

0.01368421052631579

In [11]:
behavior_log_df.groupBy('cateId').count().count()

9294

In [8]:
behavior_log_df.groupBy('brandId').count().count()

170718

In [9]:
behavior_log_df.count()

10000000

In [10]:
behavior_log_df.dropna().count()

10000000

In [11]:
behavior_log_df.groupBy(behavior_log_df.userId,behavior_log_df.cateId).pivot('btag').count().show()

+-------+------+----+----+----+----+
| userId|cateId| buy|cart| fav|  pv|
+-------+------+----+----+----+----+
| 738396|  9687|null|   1|null|  13|
| 590694|  6432|null|null|null|   3|
|1056983|  4283|null|   1|null|  31|
| 950232|  5953|null|null|null|   1|
| 386718|  9295|null|null|null|  14|
| 420528|  9494|null|null|null|  25|
| 556217|  4520|null|null|null|   6|
| 985125|  8882|   1|null|null|null|
| 199025|  6251|null|   2|null|   2|
|  99730|  4267|null|null|   1|   2|
| 244032|  6421|null|null|null|   1|
| 144924|   591|null|null|null|   4|
| 970608|  5139|null|null|null|   4|
|  95196|  4505|null|null|null|   1|
| 275462|  6250|null|null|null|   1|
|1017870|  5751|null|null|   1|   4|
| 645845|  7146|null|null|null|   1|
|  42724|  6935|   1|null|null|   5|
| 256963|  1102|   1|null|null|   1|
| 982086|  4280|null|   1|null|   9|
+-------+------+----+----+----+----+
only showing top 20 rows



In [8]:
cate_count_df = behavior_log_df.groupBy(behavior_log_df.userId, behavior_log_df.cateId).pivot("btag",["pv","fav","cart","buy"]).count()

In [10]:
cate_count_df.show(40)

+-------+------+----+----+----+----+
| userId|cateId|  pv| fav|cart| buy|
+-------+------+----+----+----+----+
| 738396|  9687|  13|null|   1|null|
| 590694|  6432|   3|null|null|null|
|1056983|  4283|  31|null|   1|null|
| 950232|  5953|   1|null|null|null|
| 386718|  9295|  14|null|null|null|
| 420528|  9494|  25|null|null|null|
| 556217|  4520|   6|null|null|null|
| 985125|  8882|null|null|null|   1|
| 199025|  6251|   2|null|   2|null|
|  99730|  4267|   2|   1|null|null|
| 244032|  6421|   1|null|null|null|
| 144924|   591|   4|null|null|null|
| 970608|  5139|   4|null|null|null|
|  95196|  4505|   1|null|null|null|
| 275462|  6250|   1|null|null|null|
|1017870|  5751|   4|   1|null|null|
| 645845|  7146|   1|null|null|null|
|  42724|  6935|   5|null|null|   1|
| 256963|  1102|   1|null|null|   1|
| 982086|  4280|   9|null|   1|null|
| 540888|   264|   1|null|null|null|
|1106063|  6940|   4|   1|null|null|
| 417953|  5947|  13|null|null|   2|
|  81888|  6736|   1|null|null|null|
|

In [11]:
spark.sparkContext.setCheckpointDir("/checkPoint/") #设置检查点 避免迭代训练的过程中 挂掉，训练几步缓存当前的参数，如果挂掉了可以从检查点加载缓存的数据

In [12]:
def process_row(r):
    #是null，就赋值0
    pv_count = r.pv if r.pv else 0.0
    fav_count = r.fav if r.fav else 0.0
    cart_count = r.cart if r.cart else 0.0
    buy_count = r.buy if r.buy else 0.0
    #每一种行为给一个系数，太大了给一个上线
    pv_score = 0.2*pv_count if pv_count<=20 else 4.0
    fav_score = 0.4*fav_count if fav_count<=20 else 8.0
    cart_score = 0.6*cart_count if cart_count<=20 else 12.0
    buy_score = 1*buy_count if buy_count<=20 else 20.0
    
    rating = pv_score+fav_score+cart_score+buy_score
    return r.userId,r.cateId,rating
    

In [13]:
cate_rating_df = cate_count_df.rdd.map(process_row).toDF(['userId','cateId','rating'])

In [14]:
cate_rating_df.show()

+-------+------+------------------+
| userId|cateId|            rating|
+-------+------+------------------+
| 738396|  9687|               3.2|
| 590694|  6432|0.6000000000000001|
|1056983|  4283|               4.6|
| 950232|  5953|               0.2|
| 386718|  9295|2.8000000000000003|
| 420528|  9494|               4.0|
| 556217|  4520|1.2000000000000002|
| 985125|  8882|               1.0|
| 199025|  6251|               1.6|
|  99730|  4267|               0.8|
| 244032|  6421|               0.2|
| 144924|   591|               0.8|
| 970608|  5139|               0.8|
|  95196|  4505|               0.2|
| 275462|  6250|               0.2|
|1017870|  5751|1.2000000000000002|
| 645845|  7146|               0.2|
|  42724|  6935|               2.0|
| 256963|  1102|               1.2|
| 982086|  4280|               2.4|
+-------+------+------------------+
only showing top 20 rows



In [15]:
### 利用spark 训练 ALS模型,checkpointInterval = 5是训练5步缓存一次
from pyspark.ml.recommendation import ALS
als = ALS(userCol = 'userId',itemCol='cateId',ratingCol = 'rating',checkpointInterval = 5)

In [16]:
model = als.fit(cate_rating_df)#模型训练可以知道每个用户对所有item的评分

In [21]:
model.recommendForAllItems(3).show(truncate=False)

+------+--------------------------------------------------------------+
|cateId|recommendations                                               |
+------+--------------------------------------------------------------+
|1580  |[[572559,2.6056695], [897038,2.4392357], [327227,2.2561812]]  |
|5300  |[[786341,11.186745], [734873,9.689989], [1049211,9.377036]]   |
|7240  |[[572559,14.611713], [1102665,9.898674], [762872,9.850235]]   |
|7340  |[[588655,11.143353], [1072885,10.365534], [156534,8.346076]]  |
|7880  |[[889855,6.030055], [572559,5.9270415], [609711,5.227521]]    |
|9900  |[[1020600,12.875811], [673610,12.728695], [800126,11.406296]] |
|1591  |[[889190,9.9742775], [792095,7.809726], [290455,6.662694]]    |
|11141 |[[762872,8.618807], [673610,8.6128025], [1102665,8.608176]]   |
|1342  |[[1020600,11.052977], [890462,10.599749], [543760,10.503686]] |
|2122  |[[572559,4.00081], [543760,3.9982512], [897453,3.4995458]]    |
|2142  |[[897453,1.8038127], [892493,1.7485108], [734873,1.71824

In [22]:
model.recommendForAllUsers(3).show(truncate = False)

+------+----------------------------------------------------------+
|userId|recommendations                                           |
+------+----------------------------------------------------------+
|463   |[[4667,3.0907784], [10411,3.000711], [2176,2.9545498]]    |
|471   |[[5595,1.3127654], [11991,1.2919976], [10149,1.1017935]]  |
|496   |[[204,0.25684738], [1861,0.24168198], [2614,0.23735447]]  |
|833   |[[5595,3.362309], [9442,3.1617591], [3612,2.7219589]]     |
|1088  |[[5595,0.5591944], [9442,0.53397405], [12000,0.45127553]] |
|1238  |[[12084,3.6526656], [9736,3.5752742], [4783,3.4106898]]   |
|1342  |[[5808,3.303752], [9442,3.2974524], [2176,3.1626403]]     |
|1580  |[[8666,0.37095994], [4783,0.35235476], [10149,0.32236597]]|
|1591  |[[9303,0.37702692], [3280,0.36623406], [204,0.3600187]]   |
|1645  |[[5595,0.7381202], [9303,0.4788307], [3280,0.44558114]]   |
|1829  |[[6375,1.7096335], [650,1.6519332], [11410,1.5553037]]    |
|1959  |[[5595,0.72765857], [7777,0.68858], [973

In [23]:
temp_df = spark.createDataFrame([[1],[2],[3]])

In [24]:
temp_df = temp_df.withColumnRenamed('_1','userId')

In [25]:
temp_df.show()

+------+
|userId|
+------+
|     1|
|     2|
|     3|
+------+



In [27]:
#模型保存
model.write().overwrite().save("/data/models/userCateRatingALSModel.obj")

In [19]:
!hls /data/models

Found 2 items
drwxr-xr-x   - root supergroup          0 2021-05-25 16:29 /data/models/CTRModel_Normal.obj
drwxr-xr-x   - root supergroup          0 2021-05-25 11:55 /data/models/userCateRatingALSModel.obj


In [ ]:
from pyspark.ml.recommendation import ALSModel
# 从hdfs加载之前存储的模型
als_model = ALSModel.load("/data/models/userCateRatingALSModel.obj")
# model.recommendForAllUsers(N) 给用户推荐TOP-N个物品
result = als_model.recommendForAllUsers(3)
result.show()

### 逻辑回归CTR预估模型创建

In [20]:
df = spark.read.csv("/data/raw_sample.csv", header=True)
df.show()    # 展示数据，默认前20条
df.printSchema()

+------+----------+----------+-----------+------+---+
|  user|time_stamp|adgroup_id|        pid|nonclk|clk|
+------+----------+----------+-----------+------+---+
|581738|1494137644|         1|430548_1007|     1|  0|
|449818|1494638778|         3|430548_1007|     1|  0|
|914836|1494650879|         4|430548_1007|     1|  0|
|914836|1494651029|         5|430548_1007|     1|  0|
|399907|1494302958|         8|430548_1007|     1|  0|
|628137|1494524935|         9|430548_1007|     1|  0|
|298139|1494462593|         9|430539_1007|     1|  0|
|775475|1494561036|         9|430548_1007|     1|  0|
|555266|1494307136|        11|430539_1007|     1|  0|
|117840|1494036743|        11|430548_1007|     1|  0|
|739815|1494115387|        11|430539_1007|     1|  0|
|623911|1494625301|        11|430548_1007|     1|  0|
|623911|1494451608|        11|430548_1007|     1|  0|
|421590|1494034144|        11|430548_1007|     1|  0|
|976358|1494156949|        13|430548_1007|     1|  0|
|286630|1494218579|        1

In [29]:
print("样本数据集总条目数：", df.count())
print("用户user总数：", df.groupBy("user").count().count())
print("广告id adgroup_id总数：", df.groupBy("adgroup_id").count().count())
print("广告展示位pid情况：", df.groupBy("pid").count().collect())
print("广告点击数据情况clk：", df.groupBy("clk").count().collect())
#可以看出大概率是不点击，广告位有两种430548_1007，和430539_1007

样本数据集总条目数： 26557961
用户user总数： 1141729
广告id adgroup_id总数： 846811
广告展示位pid情况： [Row(pid='430548_1007', count=16472898), Row(pid='430539_1007', count=10085063)]
广告点击数据情况clk： [Row(clk='0', count=25191905), Row(clk='1', count=1366056)]


In [21]:
1366056/25191905

0.05422599045209166

In [22]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType, StringType

# 打印df结构信息
df.printSchema()   
# 更改df表结构：更改列类型和列名称
raw_sample_df = df.\
    withColumn("user", df.user.cast(IntegerType())).withColumnRenamed("user", "userId").\
    withColumn("time_stamp", df.time_stamp.cast(LongType())).withColumnRenamed("time_stamp", "timestamp").\
    withColumn("adgroup_id", df.adgroup_id.cast(IntegerType())).withColumnRenamed("adgroup_id", "adgroupId").\
    withColumn("pid", df.pid.cast(StringType())).\
    withColumn("nonclk", df.nonclk.cast(IntegerType())).\
    withColumn("clk", df.clk.cast(IntegerType()))
raw_sample_df.printSchema()
raw_sample_df.show()

root
 |-- user: string (nullable = true)
 |-- time_stamp: string (nullable = true)
 |-- adgroup_id: string (nullable = true)
 |-- pid: string (nullable = true)
 |-- nonclk: string (nullable = true)
 |-- clk: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- adgroupId: integer (nullable = true)
 |-- pid: string (nullable = true)
 |-- nonclk: integer (nullable = true)
 |-- clk: integer (nullable = true)

+------+----------+---------+-----------+------+---+
|userId| timestamp|adgroupId|        pid|nonclk|clk|
+------+----------+---------+-----------+------+---+
|581738|1494137644|        1|430548_1007|     1|  0|
|449818|1494638778|        3|430548_1007|     1|  0|
|914836|1494650879|        4|430548_1007|     1|  0|
|914836|1494651029|        5|430548_1007|     1|  0|
|399907|1494302958|        8|430548_1007|     1|  0|
|628137|1494524935|        9|430548_1007|     1|  0|
|298139|1494462593|        9|430539_1007|     1|  0|


In [23]:
## 对pid进行one-hot处理
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [24]:
#利用StringIndexer 把字符串类别转换成 0 1 2 数值类别
stringIndexer = StringIndexer(inputCol = 'pid',outputCol= 'pid_feature')
string_model = stringIndexer.fit(raw_sample_df)

In [25]:
stringIndex_result = string_model.transform(raw_sample_df)

In [26]:
stringIndex_result.show()

+------+----------+---------+-----------+------+---+-----------+
|userId| timestamp|adgroupId|        pid|nonclk|clk|pid_feature|
+------+----------+---------+-----------+------+---+-----------+
|581738|1494137644|        1|430548_1007|     1|  0|        0.0|
|449818|1494638778|        3|430548_1007|     1|  0|        0.0|
|914836|1494650879|        4|430548_1007|     1|  0|        0.0|
|914836|1494651029|        5|430548_1007|     1|  0|        0.0|
|399907|1494302958|        8|430548_1007|     1|  0|        0.0|
|628137|1494524935|        9|430548_1007|     1|  0|        0.0|
|298139|1494462593|        9|430539_1007|     1|  0|        1.0|
|775475|1494561036|        9|430548_1007|     1|  0|        0.0|
|555266|1494307136|       11|430539_1007|     1|  0|        1.0|
|117840|1494036743|       11|430548_1007|     1|  0|        0.0|
|739815|1494115387|       11|430539_1007|     1|  0|        1.0|
|623911|1494625301|       11|430548_1007|     1|  0|        0.0|
|623911|1494451608|      

In [36]:
#利用OneHotEncoder 在stringIndexer的基础上 获取onehot编码，dropLast是丢掉最后一个特征，可以把dropLast改为False
encoder = OneHotEncoder(dropLast = False,inputCol = 'pid_feature',outputCol = 'pid_value')

In [37]:
result = encoder.transform(stringIndex_result)

In [38]:
result.show()

+------+----------+---------+-----------+------+---+-----------+-------------+
|userId| timestamp|adgroupId|        pid|nonclk|clk|pid_feature|    pid_value|
+------+----------+---------+-----------+------+---+-----------+-------------+
|581738|1494137644|        1|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|449818|1494638778|        3|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|914836|1494650879|        4|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|914836|1494651029|        5|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|399907|1494302958|        8|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|628137|1494524935|        9|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|298139|1494462593|        9|430539_1007|     1|  0|        1.0|(2,[1],[1.0])|
|775475|1494561036|        9|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|555266|1494307136|       11|430539_1007|     1|  0|        1.0|(2,[1],[1.0])|
|117840|1494036743|       11|430548_1007|     1|  0|

In [102]:
from pyspark.ml.linalg import SparseVector
# 参数：维度、索引列表、值列表
print(SparseVector(4, [1, 3], [3.0, 4.0]))
print(SparseVector(4, [1, 3], [3.0, 4.0]).toArray())

(4,[1,3],[3.0,4.0])
[0. 3. 0. 4.]


In [31]:
from pyspark.ml.linalg import SparseVector
# 参数：维度、索引列表、值列表
print(SparseVector(2,[1],[1.0]).toArray())   #是1
print(SparseVector(2,[0],[1.0]).toArray())   #是零

[0. 1.]
[1. 0.]


In [35]:
from pyspark.ml.linalg import SparseVector
# 参数：维度、索引列表、值列表
print(SparseVector(1,[],[]).toArray())
print(SparseVector(1,[0],[1.0]).toArray())

[0.]
[1.]


In [39]:
# StringIndexer对指定字符串列进行特征处理
stringindexer = StringIndexer(inputCol='pid', outputCol='pid_feature')

# 对处理出来的特征处理列进行，热独编码
encoder = OneHotEncoder(dropLast=False, inputCol='pid_feature', outputCol='pid_value')
# 利用管道对每一个数据进行热独编码处理
pipeline = Pipeline(stages=[stringindexer, encoder])
pipeline_model = pipeline.fit(raw_sample_df)
new_df = pipeline_model.transform(raw_sample_df)
new_df.show()

+------+----------+---------+-----------+------+---+-----------+-------------+
|userId| timestamp|adgroupId|        pid|nonclk|clk|pid_feature|    pid_value|
+------+----------+---------+-----------+------+---+-----------+-------------+
|581738|1494137644|        1|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|449818|1494638778|        3|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|914836|1494650879|        4|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|914836|1494651029|        5|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|399907|1494302958|        8|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|628137|1494524935|        9|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|298139|1494462593|        9|430539_1007|     1|  0|        1.0|(2,[1],[1.0])|
|775475|1494561036|        9|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|555266|1494307136|       11|430539_1007|     1|  0|        1.0|(2,[1],[1.0])|
|117840|1494036743|       11|430548_1007|     1|  0|

In [10]:
# 参数：维度、索引列表、值列表
print(SparseVector(2,[1],[1.0]).toArray())

[0. 1.]


In [40]:
new_df.sort('timestamp',ascending=False).show()

+------+----------+---------+-----------+------+---+-----------+-------------+
|userId| timestamp|adgroupId|        pid|nonclk|clk|pid_feature|    pid_value|
+------+----------+---------+-----------+------+---+-----------+-------------+
|177002|1494691186|   593001|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|243671|1494691186|   600195|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
| 17054|1494691184|   756665|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
| 17054|1494691184|   742741|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|488527|1494691184|   687854|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|488527|1494691184|   431082|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|488527|1494691184|   494312|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|839493|1494691183|   561681|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|839493|1494691183|   582235|430548_1007|     1|  0|        0.0|(2,[0],[1.0])|
|704223|1494691183|   624504|430539_1007|     1|  0|

In [11]:
from datetime import datetime
datetime.fromtimestamp(1494691186)

datetime.datetime(2017, 5, 13, 23, 59, 46)

In [108]:
datetime.fromtimestamp(1494691186-24*60*60)

datetime.datetime(2017, 5, 12, 23, 59, 46)

In [41]:
#训练集测试集划分
train_sample = raw_sample_df.filter(raw_sample_df.timestamp<=(1494691186-24*60*60))
#拿最后一天的数据作为测试集
test_sample = raw_sample_df.filter(raw_sample_df.timestamp>(1494691186-24*60*60))

In [42]:
train_sample.count()

23249291

In [43]:
test_sample.count()

3308670

In [44]:
#加载广告数据
df = spark.read.csv("/data/ad_feature.csv", header=True)
df.show()  

+----------+-------+-----------+--------+------+-----+
|adgroup_id|cate_id|campaign_id|customer| brand|price|
+----------+-------+-----------+--------+------+-----+
|     63133|   6406|      83237|       1| 95471|170.0|
|    313401|   6406|      83237|       1| 87331|199.0|
|    248909|    392|      83237|       1| 32233| 38.0|
|    208458|    392|      83237|       1|174374|139.0|
|    110847|   7211|     135256|       2|145952|32.99|
|    607788|   6261|     387991|       6|207800|199.0|
|    375706|   4520|     387991|       6|  NULL| 99.0|
|     11115|   7213|     139747|       9|186847| 33.0|
|     24484|   7207|     139744|       9|186847| 19.0|
|     28589|   5953|     395195|      13|  NULL|428.0|
|     23236|   5953|     395195|      13|  NULL|368.0|
|    300556|   5953|     395195|      13|  NULL|639.0|
|     92560|   5953|     395195|      13|  NULL|368.0|
|    590965|   4284|      28145|      14|454237|249.0|
|    529913|   4284|      70206|      14|  NULL|249.0|
|    54693

In [45]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

# 替换掉NULL字符串，替换掉
df = df.replace("NULL", "-1")

# 打印df结构信息
df.printSchema()   
# 更改df表结构：更改列类型和列名称
ad_feature_df = df.\
    withColumn("adgroup_id", df.adgroup_id.cast(IntegerType())).withColumnRenamed("adgroup_id", "adgroupId").\
    withColumn("cate_id", df.cate_id.cast(IntegerType())).withColumnRenamed("cate_id", "cateId").\
    withColumn("campaign_id", df.campaign_id.cast(IntegerType())).withColumnRenamed("campaign_id", "campaignId").\
    withColumn("customer", df.customer.cast(IntegerType())).withColumnRenamed("customer", "customerId").\
    withColumn("brand", df.brand.cast(IntegerType())).withColumnRenamed("brand", "brandId").\
    withColumn("price", df.price.cast(FloatType()))
ad_feature_df.printSchema()
ad_feature_df.show()

root
 |-- adgroup_id: string (nullable = true)
 |-- cate_id: string (nullable = true)
 |-- campaign_id: string (nullable = true)
 |-- customer: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)

root
 |-- adgroupId: integer (nullable = true)
 |-- cateId: integer (nullable = true)
 |-- campaignId: integer (nullable = true)
 |-- customerId: integer (nullable = true)
 |-- brandId: integer (nullable = true)
 |-- price: float (nullable = true)

+---------+------+----------+----------+-------+-----+
|adgroupId|cateId|campaignId|customerId|brandId|price|
+---------+------+----------+----------+-------+-----+
|    63133|  6406|     83237|         1|  95471|170.0|
|   313401|  6406|     83237|         1|  87331|199.0|
|   248909|   392|     83237|         1|  32233| 38.0|
|   208458|   392|     83237|         1| 174374|139.0|
|   110847|  7211|    135256|         2| 145952|32.99|
|   607788|  6261|    387991|         6| 207800|199.0|
|   375706| 

In [16]:
print("总广告条数：",df.count())   # 数据条数
_1 = ad_feature_df.groupBy("cateId").count().count()
print("cateId数值个数：", _1)
_2 = ad_feature_df.groupBy("campaignId").count().count()
print("campaignId数值个数：", _2)
_3 = ad_feature_df.groupBy("customerId").count().count()
print("customerId数值个数：", _3)
_4 = ad_feature_df.groupBy("brandId").count().count()
print("brandId数值个数：", _4)
ad_feature_df.sort("price").show()
ad_feature_df.sort("price", ascending=False).show()
print("价格高于1w的条目个数：", ad_feature_df.select("price").filter("price>10000").count())
print("价格低于1的条目个数", ad_feature_df.select("price").filter("price<1").count())

总广告条数： 846811
cateId数值个数： 6769
campaignId数值个数： 423436
customerId数值个数： 255875
brandId数值个数： 99815
+---------+------+----------+----------+-------+-----+
|adgroupId|cateId|campaignId|customerId|brandId|price|
+---------+------+----------+----------+-------+-----+
|   598024|  9970|     22467|     59048|  17554| 0.01|
|    71678|  9866|    124203|     91492|  63885| 0.01|
|   692990|  6018|    353223|    223320|     -1| 0.01|
|   169988| 10539|    238823|    221154| 211816| 0.01|
|   620285|  7043|    365821|      1960| 188191| 0.01|
|     8704|   497|    388492|     13876|     -1| 0.01|
|    42055|  9994|     43866|    113068| 123242| 0.01|
|     6636|  6703|    392038|     46239| 406713| 0.01|
|   288172|  9995|    314179|    230326| 399440| 0.01|
|   518883|  7185|    403318|     58013|     -1| 0.01|
|     2246|  9996|    413653|     60214| 182966| 0.01|
|    20397| 10539|    410958|     65726|  79971| 0.01|
|    89831| 10539|     90141|     83948| 211816| 0.01|
|    67558|  9995|     9

In [47]:
## 用户属性数据分析
df = spark.read.csv("/data/user_profile.csv", header=True)

In [48]:
df.show()

+------+---------+------------+-----------------+---------+------------+--------------+----------+---------------------+
|userid|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level |
+------+---------+------------+-----------------+---------+------------+--------------+----------+---------------------+
|   234|        0|           5|                2|        5|        null|             3|         0|                    3|
|   523|        5|           2|                2|        2|           1|             3|         1|                    2|
|   612|        0|           8|                1|        2|           2|             3|         0|                 null|
|  1670|        0|           4|                2|        4|        null|             1|         0|                 null|
|  2545|        0|          10|                1|        4|        null|             3|         0|                 null|
|  3644|       49|           6| 

In [49]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType

# 构建表结构schema对象
schema = StructType([
    StructField("userId", IntegerType()),
    StructField("cms_segid", IntegerType()),
    StructField("cms_group_id", IntegerType()),
    StructField("final_gender_code", IntegerType()),
    StructField("age_level", IntegerType()),
    StructField("pvalue_level", IntegerType()),
    StructField("shopping_level", IntegerType()),
    StructField("occupation", IntegerType()),
    StructField("new_user_class_level", IntegerType())
])
# 利用schema从hdfs加载
user_profile_df = spark.read.csv("/data/user_profile.csv", header=True, schema=schema)
user_profile_df.printSchema()
user_profile_df.show()

root
 |-- userId: integer (nullable = true)
 |-- cms_segid: integer (nullable = true)
 |-- cms_group_id: integer (nullable = true)
 |-- final_gender_code: integer (nullable = true)
 |-- age_level: integer (nullable = true)
 |-- pvalue_level: integer (nullable = true)
 |-- shopping_level: integer (nullable = true)
 |-- occupation: integer (nullable = true)
 |-- new_user_class_level: integer (nullable = true)

+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+
|userId|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level|
+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+
|   234|        0|           5|                2|        5|        null|             3|         0|                   3|
|   523|        5|           2|                2|        2|           1|             3|         1|          

In [75]:
user_profile_df.count()

1061768

In [50]:
print("分类特征值个数情况: ")
print("cms_segid: ", user_profile_df.groupBy("cms_segid").count().count())
print("cms_group_id: ", user_profile_df.groupBy("cms_group_id").count().count())
print("final_gender_code: ", user_profile_df.groupBy("final_gender_code").count().count())
print("age_level: ", user_profile_df.groupBy("age_level").count().count())
print("shopping_level: ", user_profile_df.groupBy("shopping_level").count().count())
print("occupation: ", user_profile_df.groupBy("occupation").count().count())

print("含缺失值的特征情况: ")
user_profile_df.groupBy("pvalue_level").count().show()
user_profile_df.groupBy("new_user_class_level").count().show()

t_count = user_profile_df.count()
pl_na_count = t_count - user_profile_df.dropna(subset=["pvalue_level"]).count()
print("pvalue_level的空值情况：", pl_na_count, "空值占比：%0.2f%%"%(pl_na_count/t_count*100))
nul_na_count = t_count - user_profile_df.dropna(subset=["new_user_class_level"]).count()
print("new_user_class_level(用户城市层级)的空值情况：", nul_na_count, "空值占比：%0.2f%%"%(nul_na_count/t_count*100))

分类特征值个数情况: 
cms_segid:  97
cms_group_id:  13
final_gender_code:  2
age_level:  7
shopping_level:  3
occupation:  2
含缺失值的特征情况: 
+------------+------+
|pvalue_level| count|
+------------+------+
|        null|575917|
|           1|154436|
|           3| 37759|
|           2|293656|
+------------+------+

+--------------------+------+
|new_user_class_level| count|
+--------------------+------+
|                null|344920|
|                   1| 80548|
|                   3|173047|
|                   4|138833|
|                   2|324420|
+--------------------+------+

pvalue_level的空值情况： 575917 空值占比：54.24%
new_user_class_level(用户城市层级)的空值情况： 344920 空值占比：32.49%


In [51]:
from pyspark.mllib.regression import LabeledPoint
## 数据准备成labelpoint的list，标签减1是因为预测需要的目标值是0,1,2
train_data = user_profile_df.dropna(subset=['pvalue_level']).rdd.map(lambda r: LabeledPoint(r.pvalue_level-1,[r.cms_segid, r.cms_group_id, r.final_gender_code, r.age_level, r.shopping_level, r.occupation]))

In [52]:
train_data.collect()

[LabeledPoint(0.0, [5.0,2.0,2.0,2.0,3.0,1.0]),
 LabeledPoint(1.0, [0.0,8.0,1.0,2.0,3.0,0.0]),
 LabeledPoint(1.0, [49.0,6.0,2.0,6.0,3.0,0.0]),
 LabeledPoint(1.0, [44.0,5.0,2.0,5.0,3.0,0.0]),
 LabeledPoint(0.0, [2.0,1.0,2.0,1.0,3.0,0.0]),
 LabeledPoint(1.0, [43.0,5.0,2.0,5.0,3.0,0.0]),
 LabeledPoint(1.0, [5.0,2.0,2.0,2.0,3.0,1.0]),
 LabeledPoint(1.0, [55.0,8.0,1.0,2.0,2.0,0.0]),
 LabeledPoint(1.0, [33.0,4.0,2.0,4.0,3.0,0.0]),
 LabeledPoint(1.0, [0.0,4.0,2.0,4.0,3.0,0.0]),
 LabeledPoint(1.0, [0.0,4.0,2.0,4.0,3.0,0.0]),
 LabeledPoint(0.0, [8.0,2.0,2.0,2.0,3.0,0.0]),
 LabeledPoint(0.0, [31.0,4.0,2.0,4.0,3.0,0.0]),
 LabeledPoint(1.0, [20.0,3.0,2.0,3.0,3.0,0.0]),
 LabeledPoint(1.0, [33.0,4.0,2.0,4.0,3.0,0.0]),
 LabeledPoint(1.0, [36.0,5.0,2.0,5.0,1.0,0.0]),
 LabeledPoint(1.0, [20.0,3.0,2.0,3.0,3.0,0.0]),
 LabeledPoint(1.0, [8.0,2.0,2.0,2.0,3.0,0.0]),
 LabeledPoint(1.0, [0.0,4.0,2.0,4.0,3.0,0.0]),
 LabeledPoint(0.0, [5.0,2.0,2.0,2.0,3.0,1.0]),
 LabeledPoint(1.0, [20.0,3.0,2.0,3.0,3.0,0.0]),
 L

In [54]:
#开始训练
from pyspark.mllib.tree import RandomForest
model = RandomForest.trainClassifier(train_data,3,{},5) #3代表标签类别，5代表决策树的数目

In [55]:
#来看下是否可以
model.predict([8.0,2.0,2.0,2.0,3.0,0.0])

0.0

In [56]:
#把空值填写为-1,只取出了非空的数据集
pl_na_df = user_profile_df.na.fill(-1).where('pvalue_level=-1')

In [57]:
pl_na_df.show()

+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+
|userId|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level|
+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+
|   234|        0|           5|                2|        5|          -1|             3|         0|                   3|
|  1670|        0|           4|                2|        4|          -1|             1|         0|                  -1|
|  2545|        0|          10|                1|        4|          -1|             3|         0|                  -1|
|  6211|        0|           9|                1|        3|          -1|             3|         0|                   2|
|  9293|        0|           5|                2|        5|          -1|             3|         0|                   4|
| 10812|        0|           4|         

In [58]:
def row(r):
    return r.cms_segid, r.cms_group_id, r.final_gender_code, r.age_level, r.shopping_level, r.occupation

In [59]:
rdd = pl_na_df.rdd.map(row) #拿到做预测的6个特征

In [60]:
predicts = model.predict(rdd)  #预测得到所有的值

In [61]:
#得到所有预测结果
predicts.collect()

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0

In [62]:
## 把预测好的结果 填回到有缺失的数据中，collect()是拿到数据后的list
temp = predicts.map(lambda x:int(x)).collect()

In [63]:
pdf = pl_na_df.toPandas()
#预测时是0，1,2，这次加1是变回去
import numpy as np
pdf['pvalue_level'] = np.array(temp)+1

In [64]:
na_df = spark.createDataFrame(pdf,schema=schema)

In [65]:
na_df.show()

+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+
|userId|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level|
+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+
|   234|        0|           5|                2|        5|           2|             3|         0|                   3|
|  1670|        0|           4|                2|        4|           2|             1|         0|                  -1|
|  2545|        0|          10|                1|        4|           2|             3|         0|                  -1|
|  6211|        0|           9|                1|        3|           2|             3|         0|                   2|
|  9293|        0|           5|                2|        5|           2|             3|         0|                   4|
| 10812|        0|           4|         

In [76]:
na_df.count()

575917

In [72]:
jsonString = [
"""{ "id" : 2,"city" : "a"}""",
"""{ "id" : 1,"city" : "b" }"""
]
jsonString1 = [
"""{ "id" : 2,"city" : "a"}""",
"""{ "id" : 3,"city" : "b" }""",
"""{ "id" : 4 ,"city" : "c"}"""
]
sc = spark.sparkContext
jsonSchema = StructType() \
  .add("id", LongType(),True) \
  .add("city", StringType())
jsonRDD = sc.parallelize(jsonString)
jsonRDD1 = sc.parallelize(jsonString1)
reader = spark.read.schema(jsonSchema)
jsonDF = reader.json(jsonRDD)
jsonDF.show()
jsonDF1 = reader.json(jsonRDD1)
jsonDF1.show()
jsonDF.unionAll(jsonDF1).show()

+---+----+
| id|city|
+---+----+
|  2|   a|
|  1|   b|
+---+----+

+---+----+
| id|city|
+---+----+
|  2|   a|
|  3|   b|
|  4|   c|
+---+----+

+---+----+
| id|city|
+---+----+
|  2|   a|
|  1|   b|
|  2|   a|
|  3|   b|
|  4|   c|
+---+----+



In [73]:
#把缺失的去掉，结合上咱们预测后的集合
new_user_profile_df = user_profile_df.dropna(subset=['pvalue_level']).unionAll(na_df)

In [74]:
new_user_profile_df.count()

1061768

In [36]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

# 使用热独编码转换pvalue_level的一维数据为多维，其中缺失值单独作为一个特征值

# 需要先将缺失值全部替换为数值，与原有特征一起处理
from pyspark.sql.types import StringType
user_profile_df = user_profile_df.na.fill(-1)
user_profile_df.show()

# 热独编码时，必须先将待处理字段转为字符串类型才可处理
user_profile_df = user_profile_df.withColumn("pvalue_level", user_profile_df.pvalue_level.cast(StringType()))\
    .withColumn("new_user_class_level", user_profile_df.new_user_class_level.cast(StringType()))
user_profile_df.printSchema()

# 对pvalue_level进行热独编码，求值
stringindexer = StringIndexer(inputCol='pvalue_level', outputCol='pl_onehot_feature')
encoder = OneHotEncoder(dropLast=False, inputCol='pl_onehot_feature', outputCol='pl_onehot_value')
pipeline = Pipeline(stages=[stringindexer, encoder])
pipeline_fit = pipeline.fit(user_profile_df)
user_profile_df2 = pipeline_fit.transform(user_profile_df)
# pl_onehot_value列的值为稀疏向量，存储热独编码的结果
user_profile_df2.printSchema()
user_profile_df2.show()

+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+
|userId|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level|
+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+
|   234|        0|           5|                2|        5|          -1|             3|         0|                   3|
|   523|        5|           2|                2|        2|           1|             3|         1|                   2|
|   612|        0|           8|                1|        2|           2|             3|         0|                  -1|
|  1670|        0|           4|                2|        4|          -1|             1|         0|                  -1|
|  2545|        0|          10|                1|        4|          -1|             3|         0|                  -1|
|  3644|       49|           6|         

In [37]:
stringindexer = StringIndexer(inputCol='new_user_class_level', outputCol='nucl_onehot_feature')
encoder = OneHotEncoder(dropLast=False, inputCol='nucl_onehot_feature', outputCol='nucl_onehot_value')
pipeline = Pipeline(stages=[stringindexer, encoder])
pipeline_fit = pipeline.fit(user_profile_df2)
user_profile_df3 = pipeline_fit.transform(user_profile_df2)
user_profile_df3.show()

+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+-----------------+---------------+-------------------+-----------------+
|userId|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level|pl_onehot_feature|pl_onehot_value|nucl_onehot_feature|nucl_onehot_value|
+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+-----------------+---------------+-------------------+-----------------+
|   234|        0|           5|                2|        5|          -1|             3|         0|                   3|              0.0|  (4,[0],[1.0])|                2.0|    (5,[2],[1.0])|
|   523|        5|           2|                2|        2|           1|             3|         1|                   2|              2.0|  (4,[2],[1.0])|                1.0|    (5,[1],[1.0])|
|   612|        0|           8|         

## spark ML训练逻辑回归模型

In [38]:
#先搞个小样本来看
sample_dataset = [
    (0, "male", 37, 10, "no", 3, 18, 7, 4),
    (0, "female", 27, 4, "no", 4, 14, 6, 4),
    (0, "female", 32, 15, "yes", 1, 12, 1, 4),
    (0, "male", 57, 15, "yes", 5, 18, 6, 5),
    (0, "male", 22, 0.75, "no", 2, 17, 6, 3),
    (0, "female", 32, 1.5, "no", 2, 17, 5, 5),
    (0, "female", 22, 0.75, "no", 2, 12, 1, 3),
    (0, "male", 57, 15, "yes", 2, 14, 4, 4),
    (0, "female", 32, 15, "yes", 4, 16, 1, 2),
    (0, "male", 22, 1.5, "no", 4, 14, 4, 5),
    (0, "male", 37, 15, "yes", 2, 20, 7, 2),
    (0, "male", 27, 4, "yes", 4, 18, 6, 4),
    (0, "male", 47, 15, "yes", 5, 17, 6, 4),
    (0, "female", 22, 1.5, "no", 2, 17, 5, 4),
    (0, "female", 27, 4, "no", 4, 14, 5, 4),
    (0, "female", 37, 15, "yes", 1, 17, 5, 5),
    (0, "female", 37, 15, "yes", 2, 18, 4, 3),
    (0, "female", 22, 0.75, "no", 3, 16, 5, 4),
    (0, "female", 22, 1.5, "no", 2, 16, 5, 5),
    (0, "female", 27, 10, "yes", 2, 14, 1, 5),
    (1, "female", 32, 15, "yes", 3, 14, 3, 2),
    (1, "female", 27, 7, "yes", 4, 16, 1, 2),
    (1, "male", 42, 15, "yes", 3, 18, 6, 2),
    (1, "female", 42, 15, "yes", 2, 14, 3, 2),
    (1, "male", 27, 7, "yes", 2, 17, 5, 4),
    (1, "male", 32, 10, "yes", 4, 14, 4, 3),
    (1, "male", 47, 15, "yes", 3, 16, 4, 2),
    (0, "male", 37, 4, "yes", 2, 20, 6, 4)
] 

columns = ["affairs", "gender", "age", "label", "children", "religiousness", "education", "occupation", "rating"]

In [39]:
import pandas as pd
pdf = pd.DataFrame(sample_dataset,columns=columns)

In [40]:
pdf

,affairs,gender,age,label,children,religiousness,education,occupation,rating
0,0,male,37,10.00,no,3,18,7,4
1,0,female,27,4.00,no,4,14,6,4
2,0,female,32,15.00,yes,1,12,1,4
3,0,male,57,15.00,yes,5,18,6,5
4,0,male,22,0.75,no,2,17,6,3
5,0,female,32,1.50,no,2,17,5,5
6,0,female,22,0.75,no,2,12,1,3
7,0,male,57,15.00,yes,2,14,4,4
8,0,female,32,15.00,yes,4,16,1,2
9,0,male,22,1.50,no,4,14,4,5


In [41]:
df = spark.createDataFrame(pdf)

In [42]:
df.show()

+-------+------+---+-----+--------+-------------+---------+----------+------+
|affairs|gender|age|label|children|religiousness|education|occupation|rating|
+-------+------+---+-----+--------+-------------+---------+----------+------+
|      0|  male| 37| 10.0|      no|            3|       18|         7|     4|
|      0|female| 27|  4.0|      no|            4|       14|         6|     4|
|      0|female| 32| 15.0|     yes|            1|       12|         1|     4|
|      0|  male| 57| 15.0|     yes|            5|       18|         6|     5|
|      0|  male| 22| 0.75|      no|            2|       17|         6|     3|
|      0|female| 32|  1.5|      no|            2|       17|         5|     5|
|      0|female| 22| 0.75|      no|            2|       12|         1|     3|
|      0|  male| 57| 15.0|     yes|            2|       14|         4|     4|
|      0|female| 32| 15.0|     yes|            4|       16|         1|     2|
|      0|  male| 22|  1.5|      no|            4|       14|     

In [43]:
from pyspark.ml.feature import VectorAssembler

In [44]:
# 特征选取：affairs为目标值，其余为特征值
df2 = df.select("affairs","age", "religiousness", "education", "occupation", "rating")

# 用于计算特征向量的字段
colArray2 = ["age", "religiousness", "education", "occupation", "rating"]

In [45]:
df3 = VectorAssembler().setInputCols(colArray2).setOutputCol('feautures').transform(df2)

In [46]:
df3.show(truncate = False)

+-------+---+-------------+---------+----------+------+-----------------------+
|affairs|age|religiousness|education|occupation|rating|feautures              |
+-------+---+-------------+---------+----------+------+-----------------------+
|0      |37 |3            |18       |7         |4     |[37.0,3.0,18.0,7.0,4.0]|
|0      |27 |4            |14       |6         |4     |[27.0,4.0,14.0,6.0,4.0]|
|0      |32 |1            |12       |1         |4     |[32.0,1.0,12.0,1.0,4.0]|
|0      |57 |5            |18       |6         |5     |[57.0,5.0,18.0,6.0,5.0]|
|0      |22 |2            |17       |6         |3     |[22.0,2.0,17.0,6.0,3.0]|
|0      |32 |2            |17       |5         |5     |[32.0,2.0,17.0,5.0,5.0]|
|0      |22 |2            |12       |1         |3     |[22.0,2.0,12.0,1.0,3.0]|
|0      |57 |2            |14       |4         |4     |[57.0,2.0,14.0,4.0,4.0]|
|0      |32 |4            |16       |1         |2     |[32.0,4.0,16.0,1.0,2.0]|
|0      |22 |4            |14       |4  

In [47]:
# 训练集测试集的划分
trainDF,testDF = df3.randomSplit([0.8,0.2])

In [48]:
testDF.count()

1

In [49]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()
model = lr.setLabelCol('affairs').setFeaturesCol('feautures').fit(trainDF)

In [50]:
#最左边是真实值，最后边是预测值
model.transform(testDF).show()

+-------+---+-------------+---------+----------+------+--------------------+--------------------+--------------------+----------+
|affairs|age|religiousness|education|occupation|rating|           feautures|       rawPrediction|         probability|prediction|
+-------+---+-------------+---------+----------+------+--------------------+--------------------+--------------------+----------+
|      1| 32|            4|       14|         4|     3|[32.0,4.0,14.0,4....|[0.57615844852555...|[0.64018298719625...|       0.0|
+-------+---+-------------+---------+----------+------+--------------------+--------------------+--------------------+----------+



### CTR（点击通过率）预估模型训练

In [51]:
_raw_sample_df1 = spark.read.csv("/data/raw_sample.csv", header=True)
# _raw_sample_df1.show()    # 展示数据，默认前20条
# 更改表结构，转换为对应的数据类型
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType, StringType
  
# 更改df表结构：更改列类型和列名称
_raw_sample_df2 = _raw_sample_df1.\
    withColumn("user", _raw_sample_df1.user.cast(IntegerType())).withColumnRenamed("user", "userId").\
    withColumn("time_stamp", _raw_sample_df1.time_stamp.cast(LongType())).withColumnRenamed("time_stamp", "timestamp").\
    withColumn("adgroup_id", _raw_sample_df1.adgroup_id.cast(IntegerType())).withColumnRenamed("adgroup_id", "adgroupId").\
    withColumn("pid", _raw_sample_df1.pid.cast(StringType())).\
    withColumn("nonclk", _raw_sample_df1.nonclk.cast(IntegerType())).\
    withColumn("clk", _raw_sample_df1.clk.cast(IntegerType()))
_raw_sample_df2.printSchema()
_raw_sample_df2.show()

# 样本数据pid特征处理
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

stringindexer = StringIndexer(inputCol='pid', outputCol='pid_feature')
encoder = OneHotEncoder(dropLast=False, inputCol='pid_feature', outputCol='pid_value')
pipeline = Pipeline(stages=[stringindexer, encoder])
pipeline_fit = pipeline.fit(_raw_sample_df2)
raw_sample_df = pipeline_fit.transform(_raw_sample_df2)
raw_sample_df.show()

root
 |-- userId: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- adgroupId: integer (nullable = true)
 |-- pid: string (nullable = true)
 |-- nonclk: integer (nullable = true)
 |-- clk: integer (nullable = true)

+------+----------+---------+-----------+------+---+
|userId| timestamp|adgroupId|        pid|nonclk|clk|
+------+----------+---------+-----------+------+---+
|581738|1494137644|        1|430548_1007|     1|  0|
|449818|1494638778|        3|430548_1007|     1|  0|
|914836|1494650879|        4|430548_1007|     1|  0|
|914836|1494651029|        5|430548_1007|     1|  0|
|399907|1494302958|        8|430548_1007|     1|  0|
|628137|1494524935|        9|430548_1007|     1|  0|
|298139|1494462593|        9|430539_1007|     1|  0|
|775475|1494561036|        9|430548_1007|     1|  0|
|555266|1494307136|       11|430539_1007|     1|  0|
|117840|1494036743|       11|430548_1007|     1|  0|
|739815|1494115387|       11|430539_1007|     1|  0|
|623911|1494625301|   

In [52]:
_ad_feature_df = spark.read.csv("/data/ad_feature.csv", header=True)

# 更改表结构，转换为对应的数据类型
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

# 替换掉NULL字符串
_ad_feature_df = _ad_feature_df.replace("NULL", "-1")
 
# 更改df表结构：更改列类型和列名称，campaign_id淘宝客的id
ad_feature_df = _ad_feature_df.\
    withColumn("adgroup_id", _ad_feature_df.adgroup_id.cast(IntegerType())).withColumnRenamed("adgroup_id", "adgroupId").\
    withColumn("cate_id", _ad_feature_df.cate_id.cast(IntegerType())).withColumnRenamed("cate_id", "cateId").\
    withColumn("campaign_id", _ad_feature_df.campaign_id.cast(IntegerType())).withColumnRenamed("campaign_id", "campaignId").\
    withColumn("customer", _ad_feature_df.customer.cast(IntegerType())).withColumnRenamed("customer", "customerId").\
    withColumn("brand", _ad_feature_df.brand.cast(IntegerType())).withColumnRenamed("brand", "brandId").\
    withColumn("price", _ad_feature_df.price.cast(FloatType()))
ad_feature_df.printSchema()
ad_feature_df.show()

root
 |-- adgroupId: integer (nullable = true)
 |-- cateId: integer (nullable = true)
 |-- campaignId: integer (nullable = true)
 |-- customerId: integer (nullable = true)
 |-- brandId: integer (nullable = true)
 |-- price: float (nullable = true)

+---------+------+----------+----------+-------+-----+
|adgroupId|cateId|campaignId|customerId|brandId|price|
+---------+------+----------+----------+-------+-----+
|    63133|  6406|     83237|         1|  95471|170.0|
|   313401|  6406|     83237|         1|  87331|199.0|
|   248909|   392|     83237|         1|  32233| 38.0|
|   208458|   392|     83237|         1| 174374|139.0|
|   110847|  7211|    135256|         2| 145952|32.99|
|   607788|  6261|    387991|         6| 207800|199.0|
|   375706|  4520|    387991|         6|     -1| 99.0|
|    11115|  7213|    139747|         9| 186847| 33.0|
|    24484|  7207|    139744|         9| 186847| 19.0|
|    28589|  5953|    395195|        13|     -1|428.0|
|    23236|  5953|    395195|       

In [53]:

user_profile_df3.show()

+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+-----------------+---------------+-------------------+-----------------+
|userId|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level|pl_onehot_feature|pl_onehot_value|nucl_onehot_feature|nucl_onehot_value|
+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+-----------------+---------------+-------------------+-----------------+
|   234|        0|           5|                2|        5|          -1|             3|         0|                   3|              0.0|  (4,[0],[1.0])|                2.0|    (5,[2],[1.0])|
|   523|        5|           2|                2|        2|           1|             3|         1|                   2|              2.0|  (4,[2],[1.0])|                1.0|    (5,[1],[1.0])|
|   612|        0|           8|         

In [152]:
#我们要知道对应关系，pvalue_level和pl_onehot_feature的对应
user_profile_df3.groupBy("pvalue_level").min("pl_onehot_feature").show()

+------------+----------------------+
|pvalue_level|min(pl_onehot_feature)|
+------------+----------------------+
|          -1|                   0.0|
|           3|                   3.0|
|           1|                   2.0|
|           2|                   1.0|
+------------+----------------------+



In [54]:
#以raw_sample为基准
### raw_sample 将ad_feature user_profile 数据合并起来
condition = [raw_sample_df.adgroupId == ad_feature_df.adgroupId]
_ = raw_sample_df.join(ad_feature_df,condition,'outer')

In [55]:
condition2 = [_.userId == user_profile_df3.userId]
datasets = _.join(user_profile_df3,condition2,'outer')

In [56]:
#看下结合后的数据结构
datasets.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- adgroupId: integer (nullable = true)
 |-- pid: string (nullable = true)
 |-- nonclk: integer (nullable = true)
 |-- clk: integer (nullable = true)
 |-- pid_feature: double (nullable = true)
 |-- pid_value: vector (nullable = true)
 |-- adgroupId: integer (nullable = true)
 |-- cateId: integer (nullable = true)
 |-- campaignId: integer (nullable = true)
 |-- customerId: integer (nullable = true)
 |-- brandId: integer (nullable = true)
 |-- price: float (nullable = true)
 |-- userId: integer (nullable = true)
 |-- cms_segid: integer (nullable = true)
 |-- cms_group_id: integer (nullable = true)
 |-- final_gender_code: integer (nullable = true)
 |-- age_level: integer (nullable = true)
 |-- pvalue_level: string (nullable = true)
 |-- shopping_level: integer (nullable = true)
 |-- occupation: integer (nullable = true)
 |-- new_user_class_level: string (nullable = true)
 |-- pl_onehot_feature: double (nu

In [156]:
#再看一下数据
datasets.show()

+------+----------+---------+-----------+------+---+-----------+-------------+---------+------+----------+----------+-------+------+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+-----------------+---------------+-------------------+-----------------+
|userId| timestamp|adgroupId|        pid|nonclk|clk|pid_feature|    pid_value|adgroupId|cateId|campaignId|customerId|brandId| price|userId|cms_segid|cms_group_id|final_gender_code|age_level|pvalue_level|shopping_level|occupation|new_user_class_level|pl_onehot_feature|pl_onehot_value|nucl_onehot_feature|nucl_onehot_value|
+------+----------+---------+-----------+------+---+-----------+-------------+---------+------+----------+----------+-------+------+------+---------+------------+-----------------+---------+------------+--------------+----------+--------------------+-----------------+---------------+-------------------+-----------------+
|   148|1494261938|    33926|43

In [57]:
# 剔除冗余、不需要的字段
useful_cols = [
    # 
    # 时间字段，划分训练集和测试集
    "timestamp",
    # label目标值字段
    "clk",  
    # 特征值字段
    "pid_value",       # 资源位的特征向量
    "price",    # 广告价格
    "cms_segid",    # 用户微群ID
    "cms_group_id",    # 用户组ID
    "final_gender_code",    # 用户性别特征，[1,2]
    "age_level",    # 年龄等级，1-
    "shopping_level",
    "occupation",
    "pl_onehot_value",
    "nucl_onehot_value"
]
# 筛选指定字段数据，构建新的数据集
datasets_1 = datasets.select(*useful_cols)
# 由于前面使用的是outer方式合并的数据，产生了部分空值数据，这里必须先剔除掉
datasets_1 = datasets_1.dropna()

In [58]:
from pyspark.ml.feature import VectorAssembler
#除了前两个，把后面的提取为特征
datasets_1 = VectorAssembler().setInputCols(useful_cols[2:]).setOutputCol('features').transform(datasets_1)

In [59]:
#用timestamp作为划分训练集和测试集的依据
train_datasets_1 = datasets_1.filter(datasets_1.timestamp<=(1494691186-24*60*60))
test_datasets_1 = datasets_1.filter(datasets_1.timestamp>(1494691186-24*60*60))

In [60]:
train_datasets_1.show(truncate=False)

+----------+---+-------------+------+---------+------------+-----------------+---------+--------------+----------+---------------+-----------------+--------------------------------------------------------------------------------+
|timestamp |clk|pid_value    |price |cms_segid|cms_group_id|final_gender_code|age_level|shopping_level|occupation|pl_onehot_value|nucl_onehot_value|features                                                                        |
+----------+---+-------------+------+---------+------------+-----------------+---------+--------------+----------+---------------+-----------------+--------------------------------------------------------------------------------+
|1494261938|0  |(2,[1],[1.0])|108.0 |0        |11          |1                |5        |3             |0         |(4,[0],[1.0])  |(5,[1],[1.0])    |(18,[1,2,4,5,6,7,9,14],[1.0,108.0,11.0,1.0,5.0,3.0,1.0,1.0])                    |
|1494261938|0  |(2,[1],[1.0])|1880.0|0        |11          |1                |5 

In [61]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()
model = lr.setLabelCol('clk').setFeaturesCol('features').fit(train_datasets_1)

In [62]:
# 对模型进行存储
model.write().overwrite().save("/data/models/CTRModel_Normal.obj")
# 载入训练好的模型
from pyspark.ml.classification import LogisticRegressionModel
model = LogisticRegressionModel.load("/data/models/CTRModel_Normal.obj")
# 根据测试数据进行预测
result_1 = model.transform(test_datasets_1)
# 按probability升序排列数据，probability表示预测结果的概率
# 如果预测值是0，其概率是0.9248，那么反之可推出1的可能性就是1-0.9248=0.0752，即点击概率约为7.52%
# 因为前面提到广告的点击率一般都比较低，所以预测值通常都是0，因此通常需要反减得出点击的概率
result_1.select("clk", "price", "probability", "prediction").sort("probability").show(100)

+---+-----------+--------------------+----------+
|clk|      price|         probability|prediction|
+---+-----------+--------------------+----------+
|  0|      1.0E8|[0.86822033683059...|       0.0|
|  0|      1.0E8|[0.88410456982071...|       0.0|
|  0|      1.0E8|[0.89175497615728...|       0.0|
|  1|5.5555556E7|[0.92481456390037...|       0.0|
|  0|      1.5E7|[0.93741450425576...|       0.0|
|  0|      1.5E7|[0.93757135061004...|       0.0|
|  0|      1.5E7|[0.93834723075599...|       0.0|
|  0|     1099.0|[0.93972095728799...|       0.0|
|  0|      338.0|[0.93972135008032...|       0.0|
|  0|      311.0|[0.93972136401640...|       0.0|
|  0|      300.0|[0.93972136969406...|       0.0|
|  0|      278.0|[0.93972138104939...|       0.0|
|  0|      188.0|[0.93972142750297...|       0.0|
|  0|      176.0|[0.93972143369677...|       0.0|
|  0|      168.0|[0.93972143782598...|       0.0|
|  0|      158.0|[0.93972144298748...|       0.0|
|  1|      138.0|[0.93972145331049...|       0.0|


# 离线推荐处理

In [63]:

# 这里我们只需要adgroupId、和cateId
_ = ad_feature_df.select("adgroupId", "cateId")
# 由于这里数据集其实很少，所以我们再直接转成Pandas dataframe来处理，把数据载入内存
pdf = _.toPandas()
pdf.where(pdf.cateId==11156).dropna().adgroupId

313       138953.0
314       467512.0
1661      140008.0
1666      238772.0
1669      237471.0
            ...   
843455    348435.0
843456    352273.0
846728    818681.0
846729    838953.0
846810    845337.0
Name: adgroupId, Length: 731, dtype: float64

In [64]:
np.random.choice(pdf.where(pdf.cateId==11156).dropna().adgroupId.astype(np.int64),200)

array([598297, 258576, 733900, 683715, 189810, 362888, 212226, 521847,
       271302, 316230, 760285,  81105, 468441, 341101, 373138, 370619,
       422745, 207498, 565416, 364290, 835263, 551589,  31382, 341101,
        85423, 237729,  71053, 471657, 435626, 565416, 234129, 121438,
       650941, 232586,   9933, 374667, 350638,  21093, 570388,   8927,
       468441, 293771, 234706, 233373, 109148, 260764, 565416, 232364,
       305413, 427537, 585049, 691915, 228785, 660174, 436000, 683560,
       146993, 282531,  61255, 209119, 234129, 767134, 732754, 403185,
       610668, 109673,  93659, 232586,  47556, 675555, 370619, 481589,
       768394, 594044, 545509, 629210, 110509, 263708, 373138, 732754,
       318769, 784657, 313356, 494080, 157611,  83697, 200004, 300075,
       136480, 214966, 231001,  44444, 121742, 502409,  60736, 109673,
       600931,  99717,  96408, 737225, 132623, 230645, 169899, 762867,
       245353,  99933, 138953,  15834, 157611, 120709,  56800, 627426,
      

In [65]:
from pyspark.ml.recommendation import ALSModel
# 从hdfs加载之前存储的模型
als_model = ALSModel.load("/data/models/userCateRatingALSModel.obj")
# 返回模型中关于用户的所有属性   df:   id   features
als_model.userFactors

DataFrame[id: int, features: array<float>]

In [66]:
als_model.userFactors.show(truncate=False)

+---+----------------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                                |
+---+----------------------------------------------------------------------------------------------------------------------------------------+
|7  |[0.079360895, -0.06289222, 0.055875774, -0.05377343, -0.074932456, -0.05667648, -0.06446201, 0.005515577, -0.04865939, 0.13628301]      |
|17 |[-0.1129302, -1.0777041, 0.42805904, 0.074632026, 0.020567235, 0.15854122, 0.015178426, 0.026131287, 0.86689514, 0.27400115]            |
|27 |[0.31217363, -0.35145515, 0.29314435, -0.12511745, 0.2637941, 0.22527888, 0.07589076, 0.30633223, -0.21797943, 0.37985137]              |
|37 |[-0.05762927, -0.20733033, 0.037245084, 0.018192172, -0.03849013, 0.1279327, -0.17978728, 0.19608879, 0.04094389, 0.1943658]            |

In [67]:
import numpy as np
import pandas as pd

import redis
#模型是用户对物品感兴趣的模型
# 存储用户召回，使用redis第9号数据库，类型：sets类型
client = redis.StrictRedis(host="192.168.19.137", port=6379, db=7)
#遍历als模型中的所有用户
for r in als_model.userFactors.select("id").collect():
    
    userId = r.id
    #建立用户-所有类别 的dataframe
    cateId_df = pd.DataFrame(pdf.cateId.unique(),columns=["cateId"])

    cateId_df.insert(0, "userId", np.array([userId for i in range(6769)]))
    ret = set()
    
    # 利用模型，传入datasets(userId, cateId)，为每一个用户预测每一个类别的评分 评分高的排在前面
    cateId_list = als_model.transform(spark.createDataFrame(cateId_df)).sort("prediction", ascending=False).na.drop()
    # 从前20个分类中选出500个进行召回
    for i in cateId_list.head(20):
        need = 500 - len(ret)    # 如果不足500个，那么随机选出need个广告
        ret = ret.union(np.random.choice(pdf.where(pdf.cateId==i.cateId).adgroupId.dropna().astype(np.int64), need))
        if len(ret) >= 500:    # 如果达到500个则退出
            break
    client.sadd(userId, *ret)

KeyboardInterrupt: 

In [ ]:
### 用户和物品特征的缓存
feature_cols_from_ad = [
    "price"    # 来自广告基本信息中
]

# 用户特征
feature_cols_from_user = [
    "cms_group_id",
    "final_gender_code",
    "age_level",
    "shopping_level",
    "occupation",
    "pvalue_level",
    "new_user_class_level"
]

In [ ]:
### 实时推荐处理

In [ ]:
# 数据缓存取出之后 还原成对应的onehot编码
pvalue_level_rela = {-1: 0, 3:3, 1:2, 2:1}
new_user_class_level_rela = {-1:0, 3:2, 1:4, 4:3, 2:1}
cms_group_id_rela = {
    7: 9,
    11: 6,
    3: 0,
    8: 8,
    0: 12,
    5: 3,
    6: 10,
    9: 5,
    1: 7,
    10: 4,
    4: 1,
    12: 11,
    2: 2
}
final_gender_code_rela = {1:1, 2:0}
age_level_rela = {3:0, 0:6, 5:2, 6:5, 1:4, 4:1, 2:3}
shopping_level_rela = {3:0, 1:2, 2:1}
occupation_rela = {0:0, 1:1}
pid_rela = {
    "430548_1007": 0, 
    "430549_1007": 1
}

In [ ]:
import redis
import json
import pandas as pd
from pyspark.ml.linalg import DenseVector


def create_datasets(userId, pid):
    client_of_recall = redis.StrictRedis(host="192.168.19.137", port=6379, db=9)
    client_of_features = redis.StrictRedis(host="192.168.19.137", port=6379, db=10)
    # 获取用户特征
    user_feature = json.loads(client_of_features.hget("user_features", userId).decode('utf-8'))
    
    # 获取用户召回集
    recall_sets = client_of_recall.smembers(userId)
    
    result = []
    
    # 遍历召回集
    for adgroupId in recall_sets:
        adgroupId = int(adgroupId)
        # 获取该广告的特征值
        ad_feature = json.loads(client_of_features.hget("ad_features", adgroupId).decode('utf-8'))
        
        features = {}
        features.update(user_feature)
        features.update(ad_feature)

        for k,v in features.items():
            if v is None:
                features[k] = -1

        features_col = [
            # 特征值
            "price",
            "cms_group_id",
            "final_gender_code",
            "age_level",
            "shopping_level",
            "occupation",
            "pid", 
            "pvalue_level",
            "new_user_class_level"
        ]
        '''
        "cms_group_id", 类别型特征，约13个分类 ==> 13维
        "final_gender_code", 类别型特征，2个分类 ==> 2维
        "age_level", 类别型特征，7个分类 ==>7维
        "shopping_level", 类别型特征，3个分类 ==> 3维
        "occupation", 类别型特征，2个分类 ==> 2维
        '''

        price = float(features["price"])

        pid_value = [0 for i in range(2)]#[0,0]
        cms_group_id_value = [0 for i in range(13)]
        final_gender_code_value = [0 for i in range(2)]
        age_level_value = [0 for i in range(7)]
        shopping_level_value = [0 for i in range(3)]
        occupation_value = [0 for i in range(2)]
        pvalue_level_value = [0 for i in range(4)]
        new_user_class_level_value = [0 for i in range(5)]

        pid_value[pid_rela[pid]] = 1
        cms_group_id_value[cms_group_id_rela[int(features["cms_group_id"])]] = 1
        final_gender_code_value[final_gender_code_rela[int(features["final_gender_code"])]] = 1
        age_level_value[age_level_rela[int(features["age_level"])]] = 1
        shopping_level_value[shopping_level_rela[int(features["shopping_level"])]] = 1
        occupation_value[occupation_rela[int(features["occupation"])]] = 1
        pvalue_level_value[pvalue_level_rela[int(features["pvalue_level"])]] = 1
        new_user_class_level_value[new_user_class_level_rela[int(features["new_user_class_level"])]] = 1
 #         print(pid_value)
#         print(cms_group_id_value)
#         print(final_gender_code_value)
#         print(age_level_value)
#         print(shopping_level_value)
#         print(occupation_value)
#         print(pvalue_level_value)
#         print(new_user_class_level_value)
        
        vector = DenseVector([price] + pid_value + cms_group_id_value + final_gender_code_value\
        + age_level_value + shopping_level_value + occupation_value + pvalue_level_value + new_user_class_level_value)
        
        result.append((userId, adgroupId, vector))
        
    return result


In [ ]:
create_datasets(88, "430548_1007")

In [ ]:
from pyspark.ml.classification import LogisticRegressionModel
CTR_model = LogisticRegressionModel.load("/data/models/CTRModel_AllOneHot.obj")
pdf = pd.DataFrame(create_datasets(88, "430548_1007"), columns=["userId", "adgroupId", "features"])
datasets = spark.createDataFrame(pdf)
datasets.show()

In [ ]:
prediction = CTR_model.transform(datasets).sort("probability")
prediction.show()

In [ ]:
prediction.select("adgroupId").head(20)

In [ ]:
[i.adgroupId for i in prediction.select("adgroupId").head(20)]